# NumPy入门
　　NumPy（Numerical Python 的简称）提供了高效存储和操作密集数据缓存的接口。在某些方面，NumPy 数组与Python 内置的列表类型非常相似。但是随着数组在维度上变大，NumPy数组提供了更加高效的存储和数据操作。NumPy 数组几乎是整个Python 数据科学工具生态系统的核心。<br>
## 2.1 理解Python中的数据类型
　　Python 是动态类型的语言，不同于其他静态类型语言（如C或Java）每个变量的数据类型都需要被明确地声明，在 Python 中，类型是动态推断的，这意味着可以将任何类型的数据指定给任何变量。<br>
　　这种灵活性是使 Python 和其他动态类型的语言更易用的原因之一。但是这种类型灵活性也指出了一个事实：Python 变量不仅是它们的值，还包括了关于值的类型的一些额外信息。<br>
### 2.1.1 Python整型不仅仅是一个整型
　　标准的 Python 实现是用 C 语言编写的。这意味着每一个 Python 对象都是一个聪明的伪 C 语言结构体，该结构体不仅包含其值，还有其他信
息。例如，当我们在 Python 中定义一个整型，例如 x = 10000 时，x并不是一个“原生”整型，而是一个指针，指向一个 C 语言的复合结构体，结构体里包含了一些值。查看 Python 3.4 的源代码，可以发现整型（长整型）的定义，如下所示（C 语言的宏经过扩展之后）：<br>
>struct _longobject {<br>
long ob_refcnt;<br>
PyTypeObject *ob_type;<br>
size_t ob_size;<br>
long ob_digit\[1\];<br>
};<br>

　　Python 3.4 中的一个整型实际上包括 4 个部分。<br>
>ob_refcnt 是一个引用计数，它帮助 Python 默默地处理内存的分配和回收。<br>
ob_type 将变量的类型编码。<br>
ob_size 指定接下来的数据成员的大小。<br>
ob_digit 包含我们希望 Python 变量表示的实际整型值。<br>

　　这意味着与 C 语言这样的编译语言中的整型相比，在 Python 中存储一个整型会有一些开销，正如下图所示。<br>
![C 整型和 Python 整型的区别](./images/《Python数据科学手册》_NumPy入门_C整形与Python整形区别.jpg)
　　这里 PyObject_HEAD 是结构体中包含引用计数、类型编码和其他之前提到的内容的部分。两者的差异在于，C 语言整型本质上是对应某个内存位置的标签，里面存储的字节会编码成整型。而 Python 的整型其实是一个指针，指向包含这个 Python 对象所有信息的某个内存位置，其中包括可以转换成整型的字节。由于 Python 的整型结构体里面还包含了大量额外的信息，所以 Python 可以自由、动态地编码。但是，Python 类型中的这些额外信息也会成为负担，在多个对象组合的结构体中尤其明显。<br>
### 2.1.2 Python列表不仅仅是一个列表
　　Python 列表是一个可变多元素容器，由于　Python 的动态类型特性，甚至可以创建一个异构的列表。但是为了获得这些灵活多元的类型，列表中的每一项必须包含各自的类型信息、引用计数和其他信息；也就是说，每一项都是一个完整的 Python 对象。来看一个特殊的例子，如果列表中的所有变量都是同一类型的，那么很多信息都会显得多余——将数据存储在固定类型的数组中应该会更高效。动态类型的列表和固定类型的（NumPy 式）数组间的区别如下图所示。<br>
　　在实现层面，数组基本上包含一个指向连续数据块的指针。另一方面，Python 列表包含一个指向指针块的指针，这其中的每一个指针对应一个完整的 Python 对象（如前面看到的 Python 整型）。另外，列表的优势是灵活，因为每个列表元素是一个包含数据和类型信息的完整结构体，而且列表可以用任意类型的数据填充。固定类型的 NumPy 式数组缺乏这种灵活性，但是能更有效地存储和操作数据。<br>
![Numpy数组与Python列表区别](./images/《Python数据科学手册》_NumPy入门_Numpy数组与Python列表区别.png)
### 2.1.3 Python中的固定类型数组
　　Python 提供了几种将数据存储在有效的、固定类型的数据缓存中的选项。内置的数组（array）模块（在 Python 3.3 之后可用）可以用于创
建统一类型的密集数组：<br>
>In[6]: import array<br>
L = list(range(10))<br>
A = array.array('i', L)　　　#  'i' 是一个数据类型码，表示数据为整型。<br>
A<br>
Out[6]: array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])<br>

　　更实用的是 NumPy 包中的 ndarray 对象。Python 的数组对象提供了数组型数据的有效存储，而 NumPy 为该数据加上了高效的操作。<br>
### 2.1.4 从Python列表创建数组
　　np.array 从 Python 列表创建数组。不同于 Python 列表，NumPy 要求数组必须包含同一类型的数据。如果类型不匹配，NumPy 将会向上转换（如果可行）。<br>
　　如果希望明确设置数组的数据类型，可以用 dtype 关键字。<br>
　　最后，不同于 Python 列表，NumPy 数组可以被指定为多维的。<br>
### 2.1.5 从头创建数组
　　面对大型数组的时候，用 NumPy 内置的方法从头创建数组是一种更高效的方法。<br>

|内置方法|释义|
|:--- |:---|
|np.zeros(10, dtype=int)|创建一个长度为10的数组，数组的值都是0|
|np.ones((3, 5), dtype=float)|创建一个3×5的浮点型数组，数组的值都是1|
|np.full((3, 5), 3.14)|创建一个3×5的浮点型数组，数组的值都是3.14|
|np.arange(0, 20, 2)|从0开始，到20结束，步长为2。它和内置的range()函数类似|
|np.linspace(0, 1, 5)|创建一个5个元素的数组，这5个数均匀地分配到0~1|
|np.random.random((3, 3))|创建一个3×3的、在0~1均匀分布的随机数组成的数组|
|np.random.normal(0, 1, (3, 3))|创建一个3×3的、均值为0、方差为1的正态分布的随机数数组|
|np.random.randint(0, 10, (3, 3))|创建一个3×3的、\[0, 10)区间的随机整型数组|
|np.eye(3)|创建一个3×3的单位矩阵|
|np.empty(3)|创建一个由3个整型数组成的未初始化的数组,数组的值是内存空间中的任意值|

### 2.1.6 NumPy标准数据类型
　　NumPy 数组包含同一类型的值，因此详细了解这些数据类型及其限制是非常重要的。因为 NumPy 是在 C 语言的基础上开发的，所以 C、
Fortran 和其他类似语言的用户会比较熟悉这些数据类型。<br>
　　当构建一个数组时，你可以用一个字符串参数来指定数据类型：np.zeros(10, dtype='int16')。或者用相关的 NumPy 对象来指定：
np.zeros(10, dtype=np.int16)。<br>
　　NumPy标准数据类型:

|数据类型|描述|
|:---|:---|
|bool_| 布尔值（真、True 或假、False），用一个字节存储|
|int_| 默认整型（类似于 C 语言中的 long，通常情况下是 int64 或 int32）|
|intc| 同 C 语言的 int 相同（通常是 int32 或 int64）|
|intp| 用作索引的整型（和 C 语言的 ssize_t 相同，通常情况下是 int32 或int64）|
|int8| 字节（byte，范围从–128 到 127）|
|int16| 整型（范围从–32768 到 32767）|
|int32| 整型（范围从–2147483648 到 2147483647）|
|int64| 整型（范围从–9223372036854775808 到 9223372036854775807）|
|uint8| 无符号整型（范围从 0 到 255）|
|uint16| 无符号整型（范围从 0 到 65535）|
|uint32| 无符号整型（范围从 0 到 4294967295）|
|uint64| 无符号整型（范围从 0 到 18446744073709551615）|
|float_| float64 的简化形式|
|float16| 半精度浮点型：符号比特位，5 比特位指数（exponent），10 比特位尾数（mantissa）|
|float32| 单精度浮点型：符号比特位，8 比特位指数，23 比特位尾数|
|float64| 双精度浮点型：符号比特位，11 比特位指数，52 比特位尾数|
|complex_| complex128 的简化形式|
|complex64| 复数，由两个 32 位浮点数表示|
|complex128| 复数，由两个 64 位浮点数表示|

　　还可以进行更高级的数据类型指定，例如指定高位字节数或低位字节数；更多的信息可以在 NumPy 文档（http:\/\/numpy.org\/）中查看。NumPy 也支持复合数据类型，这一点将会在 2.9 节中介绍。<br>

## 2.2 NumPy数组基础
　　Python 中的数据操作几乎等同于 NumPy 数组操作，甚至新出现的 Pandas 工具（第 3 章将介绍）也是构建在 NumPy 数组的基础之上的。本节将展示一些用 NumPy 数组操作获取数据或子数组，对数组进行分裂、变形和连接的例子。<br>
　　我们将介绍以下几类基本的数组操作：<br>
　　1、数组的属性<br>
　　　　确定数组的大小、形状、存储大小、数据类型。<br>
　　2、数组的索引<br>
　　　　获取和设置数组各个元素的值。<br>
　　3、数组的切分<br>
　　　　在大的数组中获取或设置更小的子数组。<br>
　　4、数组的变形<br>
　　　　改变给定数组的形状。<br>
　　5、数组的拼接和分裂<br>
　　　　将多个数组合并为一个，以及将一个数组分裂成多个。<br> 
### 2.2.1 NumPy数组的属性

|数组属性|描述|
|:---|:---|
|nidm|数组的维度|
|shape|数组每个维度的大小|
|size|数组的总大小|
|dtype|数组的数据类型|
|itemsize|表示每个数组元素字节大小|
|nbytes|表示数组总字节大小的属性（可认为是：itemsize 和 size 的乘积）|

### 2.2.2 数组索引：获取单个元素
　　NumPy 的索引方式，和 Python 列表一样；在多维数组中，可以用逗号分隔的索引元组获取元素；数组也可以使用负值来索引。<br> 
### 2.2.3 数组切片：获取子数组
　　正如此前用中括号获取单个数组元素，我们也可以用切片（slice）符号获取子数组，切片符号用冒号（:）表示。NumPy 切片语法和 Python 列表的标准切片语法相同。<br> 
　　注意：
　　步长值为负时，start 参数和stop 参数默认是被交换的。因此这是一种非常方便的逆序数组的方式。<br>
　　1、一维子数组<br>
　　2、多维子数组<br>
　　3、获取数组的行和列<br>
　　4、非副本视图的子数组<br>
　　　　关于数组切片有一点很重要也非常有用，那就是数组切片返回的是数组数据的视图，而不是数值数据的副本。这一点也是 NumPy 数组切片和 Python 列表切片的不同之处：在 Python 列表中，切片是值的副本。<br>
　　　　如果修改 Numpy 的子数组，将会看到原始数组也被修改了！这种默认的处理方式实际上非常有用：它意味着在处理非常大的数据集时，可以获取或处理这些数据集的片段，而不用复制底层的数据缓存。<br>
　　5、创建数组的副本
　　　　尽管数组视图有一些非常好的特性，但是在有些时候明确地复制数组里的数据或子数组也是非常有用的。可以很简单地通过Array对象的 copy() 方法实现。



### import numpy as np
x2 = np.array(range(12)).reshape(3, 4)
print("x2>\n", x2)
x2_sub = x2[:2, :2]
print("x2_sub>\n", x2_sub)
x3_sub = x2[:2, :2].copy()  # 获取数组副本
print("x3_sub>\n", x3_sub)
x2_sub[0, 0] = 99
print("new_x2_sub>\n", x2_sub)
print("x2>\n", x2)
x3_sub[0, 1] = 100
print("new_x3_sub>\n", x3_sub)
print("x2>\n", x2)     # 值被x2_sub的修改影响，而不被x3_sub的修改影响。



In [23]:
x3 = x2.reshape(4,3)
print(x3)
x3[1,1] = 100
print(x3)
print(x2)

[[99  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
[[ 99   1   2]
 [  3 100   5]
 [  6   7   8]
 [  9  10  11]]
[[ 99   1   2   3]
 [100   5   6   7]
 [  8   9  10  11]]
